In [0]:
print("Hello World")

In [0]:
customers_df = spark.read.csv(path ="/Volumes/quickstart_catalog/quickstart_schema/my_dataset/customers.csv", header=True, inferSchema=True)
customers_df.display()

In [0]:
products_df = spark.read.csv(path ="/Volumes/quickstart_catalog/quickstart_schema/my_dataset/products.csv", header=True, inferSchema=True)
products_df.display()

In [0]:
transactions_df = spark.read.csv(path ="/Volumes/quickstart_catalog/quickstart_schema/my_dataset/transactions.csv", header=True, inferSchema=True)
transactions_df.display()

In [0]:
from pyspark.sql.functions import *
customers_df = customers_df.withColumn(
    "email", regexp_replace("email", "@@", "@")
)
customers_df.display()

In [0]:
from pyspark.sql.functions import *

customers_df = customers_df.withColumn("email", when(col("email").contains("@@"),
                                                     col("email").replace("@@", "@")).otherwise(col("email")))\
                           .withColumn("dob_missing", col("dob").isNull())




In [0]:
from pyspark.sql.functions import *
# transactions_df = transactions_df.withColumn("discount", when(col("discount").isNull(), 0.0).otherwise(col("discount")))


transactions_df = transactions_df.withColumn("discount", when(col("discount").isNull(), 5).otherwise(col("discount")))
transactions_df.display()

question2 

In [0]:
from pyspark.sql.functions import *
new_df = transactions_df.join(products_df,on="product_id",how="inner")
new_df.display()


In [0]:
from pyspark.sql.functions import *
# new_df_2 = (new_df.groupBy("customer_id").agg(count("txn_id")).filter(col("count(txn_id)") > 1))
new_df_2 = (new_df.groupBy("customer_id").agg(countDistinct("txn_id")))

new_df_2.display()


In [0]:
from pyspark.sql.functions import *
new_df_3 = new_df_2.join(customers_df, on="customer_id", how="inner")

new_df_3 = (new_df_3.groupBy("city").agg(count("customer_id")))

new_df_3.display()

In [0]:
from pyspark.sql.functions import *
df_revenue = new_df.withColumn("revenue",new_df["price"] * new_df["quantity"] * (1 - new_df["discount"]))

df_category = (df_revenue.groupBy("category").agg(sum("revenue")).orderBy(desc("sum(revenue)")))
df_category.display()